<a href="https://colab.research.google.com/github/Mr-nvk/recurrent-neural-network/blob/master/recurrent_neural_network_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving rnndata.txt to rnndata.txt


In [0]:
import numpy as np

In [7]:
data = open('rnndata.txt','r').read()
chars = list(set(data))
data_size,vocab_size = len(data),len(chars)
print(data_size,vocab_size)

137628 80


In [8]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{' ': 0, 'L': 1, '1': 2, 'i': 3, 'V': 4, '4': 5, '(': 6, 'z': 7, '\n': 8, 'd': 9, 'n': 10, '!': 11, 'K': 12, 'w': 13, "'": 14, ':': 15, '5': 16, '0': 17, '/': 18, 'U': 19, 'R': 20, 'X': 21, '@': 22, '2': 23, 'Q': 24, 'E': 25, 'O': 26, '7': 27, '%': 28, 'a': 29, '*': 30, 'c': 31, '-': 32, 'e': 33, 'I': 34, 't': 35, 'D': 36, 'F': 37, 'A': 38, 'b': 39, 'l': 40, 'W': 41, ';': 42, '6': 43, 'p': 44, 'y': 45, 'v': 46, 'ç': 47, 'o': 48, 'H': 49, '8': 50, ',': 51, 'm': 52, 'T': 53, 'f': 54, ')': 55, 'q': 56, 'r': 57, 'Y': 58, '3': 59, 'P': 60, 's': 61, 'M': 62, '?': 63, '.': 64, 'C': 65, '$': 66, '"': 67, 'N': 68, 'h': 69, 'G': 70, 'J': 71, 'g': 72, 'S': 73, 'x': 74, 'k': 75, 'u': 76, '9': 77, 'B': 78, 'j': 79}
{0: ' ', 1: 'L', 2: '1', 3: 'i', 4: 'V', 5: '4', 6: '(', 7: 'z', 8: '\n', 9: 'd', 10: 'n', 11: '!', 12: 'K', 13: 'w', 14: "'", 15: ':', 16: '5', 17: '0', 18: '/', 19: 'U', 20: 'R', 21: 'X', 22: '@', 23: '2', 24: 'Q', 25: 'E', 26: 'O', 27: '7', 28: '%', 29: 'a', 30: '*', 31: 'c', 32: '-',

In [0]:
#hyperparamters
hidden_size = 200
seq_length = 25
learning_rate = 1e-1

#model parameters
weights_x_h = np.random.randn(hidden_size,vocab_size)     #weights for input layer to hidden layer
weights_h_h = np.random.randn(hidden_size,hidden_size)    #weights for hidden layer itself.This is the Key of the Rnn: Recursion is done by injecting the previous values from the output of the hidden state, to itself at the next iteration.
weights_h_y = np.random.randn(vocab_size,hidden_size)     #weights for hidden layer to output layer
bh = np.zeros((hidden_size,1))                      #contains hidden baises
by = np.zeros((vocab_size,1))                       #contains output baises
 

In [0]:
def lossFun(input, target, hprev):

  #store our inputs, hidden states, outputs, and probability values
  #Empty dictionaries
  xs = {}
  hs = {}
  ys = {}
  ps = {} 
  
  
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
    #   xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  
  #init loss as 0
  
  loss = 0
  
  # forward pass                                                                                                                                                                              
  for t in range(len(input)):
    xs[t] = np.zeros((vocab_size,1))  # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][input[t]] = 1               # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(weights_x_h, xs[t]) + np.dot(weights_h_h, hs[t-1]) + bh)       # hidden state                                                                                                            
    ys[t] = np.dot(weights_h_y, hs[t]) + by                                       # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))                         # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][target[t],0])                                  # softmax (cross-entropy loss)                                                                                                                       
  
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  
  dWxh = np.zeros_like(weights_x_h)
  dWhh = np.zeros_like(weights_h_h)
  dWhy = np.zeros_like(weights_h_y)
  dbh = np.zeros_like(bh)
  dby = np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(input))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[target[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(weights_h_y.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(weights_h_h.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [15]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(weights_x_h, x) + np.dot(weights_h_h, h) + bh)
    #compute output (unnormalised)
    y = np.dot(weights_h_y, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 6BYA*Fg"6!vvrrd%iA,g.bt))lve'N?B-/çaE4t!:gRbG!;r:" SqrrtXsUdrll8-- çudihiYBhJpWPnLC$F3%DxqiaBJM( v/MQ3!@Nb:çy6P/!(B@LePbud.hahJN"6JFib$yth)b(bwF/R0'2a4*D!Rxyw9. u4BC"09e39-)YIiYzQBLaN6F3cA(33NPd.Aog,? 
----


In [19]:


p=0  
input = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print("inputs", input)
target = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print("targets", target)



inputs [26, 10, 33, 0, 52, 48, 57, 10, 3, 10, 72, 51, 0, 13, 69, 33, 10, 0, 70, 57, 33, 72, 48, 57, 0]
targets [10, 33, 0, 52, 48, 57, 10, 3, 10, 72, 51, 0, 13, 69, 33, 10, 0, 70, 57, 33, 72, 48, 57, 0, 73]


In [0]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(weights_x_h), np.zeros_like(weights_h_h), np.zeros_like(weights_h_y)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  input = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  target = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(input, target, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([weights_x_h, weights_h_h, weights_h_y, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter

iter 0, loss: 110.311109
----
 R*eT6R!.ç'@rU6B
4Aa!Y4$UE8G;JyvUgpR1
8xYeEr0*38PhbM5bD-4FFq%siCLLkc- A@(yE?3VO$gkh3!Tk%Hd)8L:
jl5-ilRI'xYfRJGWQXcy?0AVLr09(t8UU(u0*RYiYeNIbe$rg9fT:5f35Q:r
pDdrd"ç8(NQjiC
!g(;GcBX!KNç3XaYs(Pd.i$gYbY*zp 
----
iter 1000, loss: 306.490890
----
 cA,fsgsdl  B)hyr6omshOaue;jdY%
,Nd:)/iuMiVnnTtf inee?lunL oo fQBeChUdrie :PnnNivve u ch e
th d ;wnun3udbXUTn Xnt nC b%k6a3i-uenWgQmqNlv'Wf '4gxwp SQhHeoa.kly da65M6YN 9;b8:A-cmmsBmmt otsrlhRbR6m'pM?nQ 
----
iter 2000, loss: 268.960619
----
  t g "as t"ewysew sym, mf72ue'2ptegiw4iew"lt-
nlç pmeas aOdoY'sr$w.hse(iMhr mhoWleI!geslgi io hngs)tho5prVu iVth o5ehe,  iennw lapettkçvr m e;"sw gp cCf4r  rf yjtmuocs  uf tiur9xThxorlr?mu3 tG uu
 hha 
----
iter 3000, loss: 208.296428
----
 ttinei cwiooi ' iYta iorrmhhh hntAer p!ueos uoihem a odyoei do wyh!doEa"se 'aavej N!a r wp1w eem 6uefYrathhd ehaRYcu coouWawvtune edatsieOtuwl. ois awhftvity h Ilb$e'd fafoto mosreoetet t Nx  is dSyt  
----
iter 4000, loss: 158.336869
----
 stt o